In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004141807556152344
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 16.0838042062286
200000 25.800782022344322
300000 28.02984711268944
400000 25.92509527805396
500000 28.068708007884986
600000 24.678853271881312
700000 26.640268793339683
800000 24.25250463284828
900000 23.553884581706985
1000000 11.478078026579157
1100000 21.190597664805626
1200000 23.717641223007472
1300000 22.14646996547051
1400000 23.276353310712206
1500000 24.557847946042884
1600000 23.254951513333143
1700000 23.333574230953293
1800000 25.029017933278993
1900000 24.572806903026766
2000000 7.774395686879768
2100000 27.266276830494064
2200000 29.311422378265025
2300000 28.966829812740908
2400000 30.007539145867803
2500000 27.767023546731412
2600000 27.717227652558844
2700000 28.46735229176194
2800000 29.107607704692956
2900000 25.89596100785329
3000000

29300000 23.52047865394016
29400000 23.70566360406914
29500000 23.777863742322708
29600000 9.762370283923554
29700000 22.132604247993683
29800000 22.780590688815135
29900000 22.75449259907871
30000000 22.79289491996929
30100000 21.572299424139416
30200000 20.00029256215269
30300000 9.060695853933476
30400000 19.092591499477862
30500000 21.63021230220362
30600000 20.845465428549474
30700000 21.465994547998513
30800000 22.339171201935137
30900000 22.906276872511288
31000000 17.70952232954117
31100000 15.923770441949323
31200000 22.878236459942613
31300000 24.42755278504705
31400000 23.111974796574597
31500000 17.668915738644944
31600000 14.147302522670591
31700000 22.043628785672528
31800000 22.3679928809149
31900000 23.37024055660512
32000000 21.332602799258083
32100000 21.141367476348126
32200000 18.13148687412001
32300000 9.633605748842207
32400000 20.046272313440518
32500000 19.759162519805578
32600000 22.740214402859994
32700000 22.11522445149925
32800000 22.932471736652193
32900000

59000000 19.80996773829721
59100000 11.337719059903861
59200000 17.46116524005421
59300000 18.202408293304906
59400000 18.55478240033416
59500000 18.344517220152348
59600000 8.146922638117784
59700000 17.972962123011342
59800000 18.339158040285596
59900000 17.95965904246872
60000000 17.84388728697826
60100000 10.220101057290545
60200000 18.68158782052811
60300000 20.336699979799857
60400000 20.547048477569074
60500000 20.829580357990096
60600000 21.731797098982163
60700000 19.78801602721686
60800000 20.085441657966392
60900000 19.529375413038792
61000000 20.64726712816543
61100000 7.932576092441748
61200000 17.412416642891426
61300000 18.85080356814608
61400000 18.73557075144374
61500000 16.67377023625028
61600000 9.506912044872891
61700000 18.260032966413313
61800000 18.705095067580807
61900000 17.55806156433408
62000000 14.659544700100358
62100000 14.142995537888066
62200000 19.533641615233332
62300000 20.428320761421762
62400000 20.37016137938755
62500000 21.047264821515874
62600000

88800000 16.68249330777209
88900000 6.160091315156394
89000000 16.01745062752083
89100000 15.450282121035276
89200000 8.931890812867179
89300000 16.06198528324581
89400000 16.07061278216319
89500000 15.649996438482402
89600000 16.958575382552343
89700000 17.245163770995923
89800000 17.653529394719566
89900000 17.48343407991025
90000000 17.349538416364243
90100000 16.52497125468357
90200000 16.092582955024277
90300000 16.83204747331647
90400000 14.958186946108851
90500000 15.000678650673708
90600000 8.709064633039388
90700000 14.918735042829846
90800000 13.95078548106916
90900000 8.193524699366764
91000000 14.921291494836401
91100000 8.625942771833518
91200000 15.518106301876456
91300000 16.00179970586442
91400000 15.602199240630886
91500000 16.075901222004543
91600000 16.601092113239503
91700000 17.038630254175473
91800000 16.933095823284724
91900000 17.997565226743344
92000000 17.973607135744597
92100000 17.599501089800867
92200000 16.416025375375675
92300000 16.74267588741715
9240000

117900000 1.5479586974897026
118000000 1.904295863631093
118100000 1.5200636588819258
118200000 1.576850512503904
118300000 1.5727712010651065
118400000 1.5479818717804152
118500000 1.7892773509270787
118600000 1.6480292980893447
118700000 1.615828103642112
118800000 1.754486308156405
118900000 1.6707881785751164
119000000 1.613768732918789
119100000 1.7520612402225615
119200000 1.671050512394109
119300000 1.653636177041734
119400000 1.5263532965772368
119500000 1.5188437379801953
119600000 1.6158239526624236
119700000 1.5917539878572144
119800000 1.4893407990061338
119900000 1.70847754640755
120000000 1.5949529504359958
120100000 1.9153068788886611
120200000 1.8668497523141265
120300000 1.6843242957983793
120400000 1.7625448762191764
120500000 1.626741169245945
120600000 1.5704855299847125
120700000 1.5559100183865433
120800000 1.6702331454744817
120900000 1.5765559408007743
121000000 1.4738132755001743
121100000 1.618083424804689
121200000 1.6564915323499323
121300000 1.5476992086644

...

204300000 8.357527589041334
204400000 7.901888252522724
204500000 8.996718391211267
204600000 8.58707184463833
204700000 9.309181643293147
204800000 4.738817318790573
204900000 9.25543645718492
205000000 9.364314804322332
205100000 9.125211046312117
205200000 9.845969084487246
205300000 8.917233624544755
205400000 5.774943396681969
205500000 7.665535355021459
205600000 9.296709076570249
205700000 8.529373412320574
205800000 8.021169347364737
205900000 8.178245886105751
206000000 7.479576722330879
206100000 2.7743098132445234
206200000 8.29519153830699
206300000 7.51268448119528
206400000 8.538210723756338
206500000 7.984470738590779
206600000 8.578106533433278
206700000 8.088714898407295
206800000 4.243013616828327
206900000 8.941994478162778
207000000 8.406253314600992
207100000 9.092757842315878
207200000 8.311664900907884
207300000 7.357392693590432
207400000 6.0904419488332575
207500000 8.518463753461823
207600000 8.606014782858432
207700000 7.866280212615881
207800000 8.0578326013

233600000 6.648247049650762
233700000 6.397772461714759
233800000 6.356125778600241
233900000 6.313858826825056
234000000 6.606058771313479
234100000 6.438435470024645
234200000 6.625797579882474
234300000 4.194120514926678
234400000 5.495627561409111
234500000 3.404597505195702
234600000 6.576229469296829
234700000 6.513726981149504
234800000 6.45347140341192
234900000 6.588281030360796
235000000 6.079206140185986
235100000 6.145708599801469
235200000 6.407394821508164
235300000 6.283531461695038
235400000 2.245823071231812
235500000 6.553551977280817
235600000 6.335171615075789
235700000 6.249446335389836
235800000 6.280842642982647
235900000 6.616750162242996
236000000 6.465774896102981
236100000 6.483406212234002
236200000 6.479202691520759
236300000 2.828976210882253
236400000 4.200005182792574
236500000 5.750107206596868
236600000 6.36817318097057
236700000 6.61174209842123
236800000 6.621166481411959
236900000 6.3077482699755745
237000000 6.126149136102307
237100000 6.2746895174

262300000 2.093094053433212
262400000 2.0264973655509153
262500000 2.6309744871215894
262600000 1.2847158055332104
262700000 0.5010755706555307
262800000 0.4943344807484389
262900000 1.2712655735686063
263000000 2.794973275649222
263100000 2.044512908444685
263200000 2.3596263380622196
263300000 2.3376380948937148
263400000 2.1039469452081323
263500000 2.3475704720201565
263600000 2.264279365137409
263700000 2.7281355854712666
263800000 2.326319839433015
263900000 2.4133960551340956
264000000 2.1298722781357586
264100000 2.1658325593288503
264200000 2.0068730478659274
264300000 2.2124043534917814
264400000 2.284695368377128
264500000 1.7403149276170373
264600000 0.8568006025439701
264700000 0.4772688150770168
264800000 0.6698811249498704
264900000 1.6851505777174411
265000000 2.512679335484484
265100000 2.2570615685260433
265200000 1.9485313705591838
265300000 2.056285013016069
265400000 2.100415613064371
265500000 2.245132656683856
265600000 2.264059807060878
265700000 2.5444069125211

290800000 1.1613804503195424
290900000 1.1382399341931477
291000000 1.2805518531386137
291100000 1.129712941607472
291200000 1.2154293699132968
291300000 1.0485870320736568
291400000 1.0751341637935057
291500000 1.0931194841837653
291600000 0.9920560854828039
291700000 0.7706428588679758
291800000 1.1119935245870973
291900000 0.5062711860902787
292000000 0.22512202638913925
292100000 0.2368586294347326
292200000 0.5573505722314138
292300000 1.1217489364699385
292400000 0.9203153452917144
292500000 1.0851065570422491
292600000 1.1476479299135545
292700000 0.9128013247608376
292800000 1.0273977871773718
292900000 0.9812351358414496
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 1.168254796537413
293100000 0.8389987782181124
293200000 0.8203528363332185
293300000 0.8527938935769837
293400000 0.7859206030241032
293500000 0.729863234907761
293600000 0.6963337806701156
293700000 0.7281289500541865
293800000 0.7251803664757669
293900000 0.5523355965416212
2940

best so far: 0
type: [1, 1, 25, 5, 1] 125
cases of this type: 78125
317200000 0.01469861507152054
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.0201116276326504
317400000 0.051731575521512835
317500000 0.0476227914778259
317600000 0.048630600709162815
317700000 0.054183729861140256
317800000 0.040462624878066115
317900000 0.03629536212969621
318000000 0.041596991877657176
318100000 0.024455074919501277
318200000 0.009906201395967934
318300000 0.012210749284019074
318400000 0.008254658069532447
318500000 0.0333386688664231
318600000 0.03680755538743535
318700000 0.02785199874403212
318800000 0.021221996510799725
318900000 0.03946662314085365
319000000 0.036135484977733424
319100000 0.0207138882587413
319200000 0.02032448004903992
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.012839674771963225
319400000 0.006424944513492451
319500000 0.00753219163150